In [0]:
from google.colab import drive
drive.mount('/gdrive')

#CSV파일로 변환해서 gdrive에 저장 하기 위해서!!

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import urllib.request
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import parse, Element
import pandas as pd

In [0]:
def query_sender(sch_date, sch_airport, sch_HH):
    url = 'http://openapi.airport.co.kr/service/rest/dailyExpectPassenger/dailyExpectPassenger?' 
    code_info = 'schDate=' + sch_date + '&schAirport=' + sch_airport + '&schTof=D' + '&schHH=' + sch_HH
    key = '&serviceKey=' + 'tQXCm87rj3LYrS5OsDyH5djylmBYjBmQyUoTSyj1jCkemuAzKijFz5ozM9fzZC5ZE2dJmjFBYyRqrjOiHvyKPA%3D%3D' 

    #key부분 각자 공공데이터포털에서 발급받은 개인 인증키로 변경하기!!
    
    request = urllib.request.Request(url+code_info+key)
    request.get_method=lambda: 'GET'
    response_body=urllib.request.urlopen(request).read()
    u = str(response_body, "utf-8")
    return u
  




def xml_to_item_list(xml_string):
    item_list = []
    root = ET.fromstring(xml_string)
    elements = root.findall('body/items/item')
    for item in elements:
        item_list.append(item.find('arp').text)
        item_list.append(item.find('sdt').text)
        item_list.append(item.find('hh').text)
        item_list.append(item.find('pct').text)
        item_list.append(item.find('pcg').text)
        item_list.append(item.find('congestYn').text)

    return item_list

In [0]:
def congestion_maker(airport, year, month, end_day): #airport, year, month = str / end_day = int
    total_list = []
    for i in range(1, end_day+1): # i = 날짜
        for j in range(6, 22): # j = 시간
            xml_string_tmp = query_sender(year + month + str(i).zfill(2), airport, str(j).zfill(2))
            sch_list_tmp = xml_to_item_list(xml_string_tmp)
            total_list.append(sch_list_tmp)
    return total_list

In [0]:
col = ['AIRPORT', 'DATE', 'HOUR', 'PASSENGER', 'GROUP_PASSNEGER', 'CONGESTION']

In [0]:
GMP_201702 = congestion_maker('GMP', '2017', '02', 28) # ('공항명', '연도', '월', 그 달의 마지막 날짜) 이렇게 집어넣기! / 김포 = 'GMP', 제주 = 'CJU', 김해 = 'PUS'

In [0]:
GMP_201702_data = pd.DataFrame(GMP_201702, columns = col)

In [0]:
GMP_201702_data.to_csv('/gdrive/My Drive/bigcontest/congestion_data/GMP_201702.csv')

In [0]:
GMP_201702_data

,AIRPORT,DATE,HOUR,PASSENGER,GROUP_PASSNEGER,CONGESTION
0,GMP,20170201,06,1689,35,N
1,GMP,20170201,07,1548,24,N
2,GMP,20170201,08,1796,65,N
3,GMP,20170201,09,1544,30,N
4,GMP,20170201,10,1338,31,N
5,GMP,20170201,11,1724,44,N
6,GMP,20170201,12,1315,54,N
7,GMP,20170201,13,1712,13,N
8,GMP,20170201,14,1519,56,N
9,GMP,20170201,15,1356,166,N
